# Sentiment analysis of YELP reviews

In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

## DATA PREPROCESSING

## Step 1: Stemming 

In [7]:
df = pd.read_csv('/Users/ishansharma/Downloads/yelp_review_full_csv/train.csv')

In [8]:
df.head()

,5,"dr. goldberg offers everything i look for in a general practitioner. he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first. really, what more do you need? i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."
0,2,"Unfortunately, the frustration of being Dr. Go..."
1,4,Been going to Dr. Goldberg for over 10 years. ...
2,4,Got a letter in the mail last week that said D...
3,1,I don't know what Dr. Goldberg was like before...
4,5,Top notch doctor in a top notch practice. Can'...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649999 entries, 0 to 649998
Data columns (total 2 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Non-Null Count   Dtype 
---  ------                                                                                                                                                                                                                                                                                                                          

### Imports

In [13]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string
from nltk.corpus import stopwords
from unidecode import unidecode
from langdetect import detect
from nltk.stem import SnowballStemmer
from googletrans import Translator

In [16]:
import nltk
nltk.download('all')
stopwords = set(stopwords.words("english"))
exclude = set(string.punctuation)
translate = Translator()
sno = SnowballStemmer("english")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package cess_cat is

[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-date!
[nltk_data]    | Downloading package universal_treebanks_v20 to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package universal_treebanks_v20 is already up-to-
[nltk_data]    |       date!
[nltk_data]    | Downloading package verbnet to
[nltk_data]    |     /Users/ishansharma/nltk_data...
[nltk_data]    |   Package verbnet is already up-to-date!
[nltk_data]    | Downloading package verbnet3 to
[nltk_data]    |     /Users/

In [17]:
def get_words(sent):
    sent = sent.lower()
    temp = ''
    for ch in sent:
        if ch not in exclude:
            temp +=ch
        else:
            temp+=''
    arr = word_tokenize(temp)
    sent = ""
    for i in arr:
        if i!='' and i!=' ' and (i not in stopwords):
            sent +=i+' '
    return sent

In [39]:
inp = pd.Series([df.empty])
inp_processed = []
err = 0

In [40]:
for i in tqdm(range(len(inp))):
    #sent = unidecode(inp[i].decode('utf-8'))
    sent = inp[i]
    if detect(sent) != 'en':
        try:
            sent = translate.translate(sent).text
        except:
            err +=1
    sent = get_words(sent)
    stemmed = ''
    for j in sent.split(" "):
        stemmed+= sno.stem(j)+" "
    df.loc[i,'text'] = stemmed.encode('utf-8')

  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: cannot use a string pattern on a bytes-like object

In [41]:
df.to_csv("stemmed_data.csv")

## Step 2: Tokenization

In [44]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import multiprocessing
cores = multiprocessing.cpu_count()
from sklearn import utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [45]:
df = pd.read_csv("stemmed_data.csv")

In [74]:
df.head()

,Unnamed: 0,5,"dr. goldberg offers everything i look for in a general practitioner. he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first. really, what more do you need? i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."
0,0,2,"Unfortunately, the frustration of being Dr. Go..."
1,1,4,Been going to Dr. Goldberg for over 10 years. ...
2,2,4,Got a letter in the mail last week that said D...
3,3,1,I don't know what Dr. Goldberg was like before...
4,4,5,Top notch doctor in a top notch practice. Can'...


In [103]:
x = pd.Series([df])
y=[]

In [104]:
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

ValueError: Found input variables with inconsistent numbers of samples: [1, 0]

In [96]:
print("Total Negative% Moderate% Positive%")
print(len(x_train),(len(x_train[y_train == 0]) / (len(x_train)))*100,(len(x_train[y_train == 1]) / (len(x_train)))*100,(len(x_train[y_train == 2]) / (len(x_train)))*100)
print(len(x_validation),(len(x_validation[y_validation == 0]) / (len(x_validation)))*100,(len(x_validation[y_validation == 1]) / (len(x_validation)))*100,(len(x_validation[y_validation == 2]) / (len(x_validation)*1.))*100)
print(len(x_test),(len(x_test[y_test == 0]) / (len(x_test)))*100,(len(x_test[y_test == 1]) / (len(x_test)))*100,(len(x_test[y_test == 2]) / (len(x_test))*100))

Total Negative% Moderate% Positive%


NameError: name 'x_train' is not defined

In [ ]:
x_train[0]

In [ ]:
y_train[0]

In [ ]:
def labelize(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [ ]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize(all_x, 'all')

In [ ]:
model = Word2Vec(sg=1, size=300, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model.build_vocab([x.words for x in tqdm(all_x_w2v)])

In [ ]:
%%time
for epoch in range(10):
    model.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

In [ ]:
model.save('w2v_model_sg.word2vec')

In [ ]:
model = Word2Vec(sg=0, size=300, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model.build_vocab([x.words for x in tqdm(all_x_w2v)])

In [ ]:
%%time
for epoch in range(10):
    model.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

In [80]:
model.save('w2v_model_cbow.word2vec') 

NameError: name 'model' is not defined

CBOW means continuous bag of words. SG means Skip Gram. 
With a corpus, CBOW model predicts the current word from a window of surrounding context words, while Skip-gram model predicts surrounding context words given the current word.


For example, let's say we have the following sentence: "I love dogs". CBOW model tries to predict the word "love" when given "I", "dogs" as inputs, on the other hand, Skip-gram model tries to predict "I", "dogs" when given the word "love" as input.

In [ ]:
from  gensim.models import KeyedVectors
model_sg = KeyedVectors.load('w2v_model_sg.word2vec')
model_cbow = KeyedVectors.load('w2v_model_cbow.word2vec')

In [ ]:
len(model_sg.wv.vocab.keys())

In [ ]:
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(x_train)

Tokenizing Train data

In [ ]:
sequences = tokenizer.texts_to_sequences(x_train)
x_train_seq = pad_sequences(sequences, maxlen=80)
print('Shape of data tensor:', x_train_seq.shape)
partial_data = pd.DataFrame({"text_vectors":np.ndarray.tolist(x_train_seq),'label':y_train},dtype = 'object')
partial_data.to_pickle("train_tokanized.pkl")

Tokenizing Validation data

In [ ]:
sequences = tokenizer.texts_to_sequences(x_validation)
x_validation_seq = pad_sequences(sequences, maxlen=80)
print('Shape of data tensor:', x_validation_seq.shape)
partial_data = pd.DataFrame({"text_vectors":np.ndarray.tolist(x_validation_seq),'label':y_validation},dtype = 'object')
partial_data.to_pickle("validation_tokanized.pkl")

Tokenizing Test data

In [ ]:
sequences = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences, maxlen=80)
print('Shape of data tensor:', x_test_seq.shape)
partial_data = pd.DataFrame({"text_vectors":np.ndarray.tolist(x_test_seq),'label':y_test},dtype = 'object')
partial_data.to_pickle("test_tokanized.pkl")

In [ ]:
embeddings_index = {}
for w in model_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_cbow.wv[w],model_sg.wv[w])
print('Found %s word vectors.' % len(embeddings_index))
num_words  = 30000
embedding_matrix = np.zeros((num_words, 600))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes = 3)
y_validation = to_categorical(y_validation,num_classes = 3)
y_test = to_categorical(y_test,num_classes = 3)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
seed = 7
import keras
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Flatten , Conv2D, MaxPooling2D, Reshape, ConvLSTM2D
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
keras.backend.clear_session()
model = Sequential()
e = Embedding(30000, 600,  input_length=80, trainable=True) #weights=[embedding_matrix]
model.add(e)
model.add(Reshape((1,80,600),input_shape=(100000,)))
model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu',))
model.add(MaxPooling2D(pool_size=(3, 3), padding='same',strides = 2))
model.add(Dropout(0.3))
model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), padding='same',strides = 2))              
model.add(Dropout(0.3))
model.add(Flatten())
#model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint
filepath="CNN_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [ ]:
model.fit(x_train_seq, y_train, validation_data=(x_validation_seq, y_validation), epochs=3, batch_size=8, callbacks = [checkpoint])

# Testing

In [ ]:
from keras.models import load_model
loaded_CNN_model = load_model('CNN_weights.h5')

In [ ]:
loaded_CNN_model.summary()

In [ ]:
#To plot model
from keras.utils.vis_utils import plot_model
plot_model(loaded_CNN_model, to_file='CNN_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
loaded_CNN_model.evaluate(x=x_test_seq, y=y_test)

In [ ]:
y_predict = loaded_CNN_model.predict(x=x_test_seq)

In [ ]:
confusion_matrix(np.argmax(y_test,axis = 1),np.argmax(y_predict,axis = 1))

In [ ]:
classification_report(np.argmax(y_test,axis = 1),np.argmax(y_predict,axis = 1))